# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2 Scraping

**Harvard University**<br>
**Summer 2018**<br>
**Instructors:** Pavlos Protopapas and Kevin Rader <br>
**Lab Instructors:** Rahul Dave <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh and Pavlos Protopaps

---

In [0]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn.apionly as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

/Users/volam/anaconda3/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
import time, requests

In this lab, we'll scrape Goodread's Best Books list:

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 .

We'll walk through scraping the list pages for the book names/urls

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Exploring the Web pages and downloading them</li>
<li> Parse the page, extract book urls </li>
<li> Parse a book page, extract book properties </li>
<li> Set up a pipeline for fetching and parsing</li>
</ol>

## Learning Goals

Understand the structure of a web page. Use Beautiful soup to scrape content from these web pages.

*This lab corresponds to lectures 2, 3 and 4 and maps on to homework 1 and further.*

## 1. Exploring the web pages and downloading them

We're going to see the structure of Goodread's best books list. We'll use the Developer tools in chrome, safari and firefox have similar tools available

![](https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/labs/lab2/notebook/solutions/images/goodreads1.png?raw=1)

To getch this page we use the `requests` module. But are we allowed to do this? Lets check:

https://www.goodreads.com/robots.txt

Yes we are.

In [3]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
print(url)
page = requests.get(url)

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1


We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found , ok. (See lecture notes.)

In [4]:
page.status_code # 200 is good

200

Let us write a loop to fetch 2 pages of "best-books" from goodreads. Notice the use of a format string. This is an example of old-style python format strings

In [5]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
for i in range(1,501):
    bookpage=str(i)
    stuff=requests.get(URLSTART+BESTBOOKS+bookpage)
    filetowrite="best_books_ever/page"+ '%02d' % i + ".html"
    print("FTW", filetowrite)
    fd=open(filetowrite,"w")
    fd.write(stuff.text)
    fd.close()
    time.sleep(1)

FTW best_books_ever/page01.html
FTW best_books_ever/page02.html
FTW best_books_ever/page03.html
FTW best_books_ever/page04.html
FTW best_books_ever/page05.html
FTW best_books_ever/page06.html
FTW best_books_ever/page07.html
FTW best_books_ever/page08.html
FTW best_books_ever/page09.html
FTW best_books_ever/page10.html
FTW best_books_ever/page11.html
FTW best_books_ever/page12.html
FTW best_books_ever/page13.html
FTW best_books_ever/page14.html
FTW best_books_ever/page15.html
FTW best_books_ever/page16.html
FTW best_books_ever/page17.html
FTW best_books_ever/page18.html
FTW best_books_ever/page19.html
FTW best_books_ever/page20.html
FTW best_books_ever/page21.html
FTW best_books_ever/page22.html
FTW best_books_ever/page23.html
FTW best_books_ever/page24.html
FTW best_books_ever/page25.html
FTW best_books_ever/page26.html
FTW best_books_ever/page27.html
FTW best_books_ever/page28.html
FTW best_books_ever/page29.html
FTW best_books_ever/page30.html
FTW best_books_ever/page31.html
FTW best

FTW best_books_ever/page253.html
FTW best_books_ever/page254.html
FTW best_books_ever/page255.html
FTW best_books_ever/page256.html
FTW best_books_ever/page257.html
FTW best_books_ever/page258.html
FTW best_books_ever/page259.html
FTW best_books_ever/page260.html
FTW best_books_ever/page261.html
FTW best_books_ever/page262.html
FTW best_books_ever/page263.html
FTW best_books_ever/page264.html
FTW best_books_ever/page265.html
FTW best_books_ever/page266.html
FTW best_books_ever/page267.html
FTW best_books_ever/page268.html
FTW best_books_ever/page269.html
FTW best_books_ever/page270.html
FTW best_books_ever/page271.html
FTW best_books_ever/page272.html
FTW best_books_ever/page273.html
FTW best_books_ever/page274.html
FTW best_books_ever/page275.html
FTW best_books_ever/page276.html
FTW best_books_ever/page277.html
FTW best_books_ever/page278.html
FTW best_books_ever/page279.html
FTW best_books_ever/page280.html
FTW best_books_ever/page281.html
FTW best_books_ever/page282.html
FTW best_b

## 2. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [6]:
from bs4 import BeautifulSoup

In [15]:
books = []
for i in range(1, 501):
    stri = '%02d' % i
    filetoread="best_books_ever/page"+ stri + '.html'
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, 'html.parser')
    for e in soup.select('.bookTitle'):
        book_id_name = e['href'].split('/')[3]
        if '.' in book_id_name:
            book_id = int(book_id_name.split('.')[0])
        else:
            book_id = int(book_id_name.split('-')[0])
        books.append(book_id)
print(books)

FTW best_books_ever/page01.html
FTW best_books_ever/page02.html
FTW best_books_ever/page03.html
FTW best_books_ever/page04.html
FTW best_books_ever/page05.html
FTW best_books_ever/page06.html
FTW best_books_ever/page07.html
FTW best_books_ever/page08.html
FTW best_books_ever/page09.html
FTW best_books_ever/page10.html
FTW best_books_ever/page11.html
FTW best_books_ever/page12.html
FTW best_books_ever/page13.html
FTW best_books_ever/page14.html
FTW best_books_ever/page15.html
FTW best_books_ever/page16.html
FTW best_books_ever/page17.html
FTW best_books_ever/page18.html
FTW best_books_ever/page19.html
FTW best_books_ever/page20.html
FTW best_books_ever/page21.html
FTW best_books_ever/page22.html
FTW best_books_ever/page23.html
FTW best_books_ever/page24.html
FTW best_books_ever/page25.html
FTW best_books_ever/page26.html
FTW best_books_ever/page27.html
FTW best_books_ever/page28.html
FTW best_books_ever/page29.html
FTW best_books_ever/page30.html
FTW best_books_ever/page31.html
FTW best

FTW best_books_ever/page253.html
FTW best_books_ever/page254.html
FTW best_books_ever/page255.html
FTW best_books_ever/page256.html
FTW best_books_ever/page257.html
FTW best_books_ever/page258.html
FTW best_books_ever/page259.html
FTW best_books_ever/page260.html
FTW best_books_ever/page261.html
FTW best_books_ever/page262.html
FTW best_books_ever/page263.html
FTW best_books_ever/page264.html
FTW best_books_ever/page265.html
FTW best_books_ever/page266.html
FTW best_books_ever/page267.html
FTW best_books_ever/page268.html
FTW best_books_ever/page269.html
FTW best_books_ever/page270.html
FTW best_books_ever/page271.html
FTW best_books_ever/page272.html
FTW best_books_ever/page273.html
FTW best_books_ever/page274.html
FTW best_books_ever/page275.html
FTW best_books_ever/page276.html
FTW best_books_ever/page277.html
FTW best_books_ever/page278.html
FTW best_books_ever/page279.html
FTW best_books_ever/page280.html
FTW best_books_ever/page281.html
FTW best_books_ever/page282.html
FTW best_b

In [21]:
best_books = pd.DataFrame({'book_id': books})

In [22]:
best_books

,book_id
0,2767052
1,2
2,2657
3,1885
4,41865
5,19063
6,7613
7,11127
8,30
9,18405


In [23]:
import pandas as pd
from goodreads import client

In [24]:
goodreads_key = 't6mIkabukH29jAey0381yA'
goodreads_secret = '6IWvvO5CNFIAqUragee2Bb5HkEOvxIYqSXeXdFSHvM'
good_client = client.GoodreadsClient(goodreads_key, goodreads_secret)

In [26]:
test = good_client.book(10)

In [29]:
features = ['title', 'authors', 'description', 'average_rating', 'rating_dist', 'ratings_count',
           'publication_date', 'publisher']

In [ ]:
book_df = {}
book_df['book_id'] = books

for i in range(len(features)):
    book_df[features[i]] = []
for book_id in books:
    try:
        book = good_client.book(book_id)
        for i in range(len(features)):
            book_df[features[i]].append(getattr(book, features[i]))
    except:
        print(book_id)
        pass

10964693
6178648
263985
64218
210404
16220184
10412
152519
754771
6017769
93722
924194
87348
4385897


![](https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/labs/lab2/notebook/solutions/images/goodreads2.png?raw=1)

## 3. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [0]:
furl=URLSTART+bookdict['02'][0]
furl

'https://www.goodreads.com/book/show/5470.1984'

![](https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/labs/lab2/notebook/solutions/images/goodreads3.png?raw=1)

In [0]:
fstuff=requests.get(furl)
print(fstuff.status_code)

200


In [0]:
d=BeautifulSoup(fstuff.text, 'html.parser')

In [0]:
d.prettify()

'<!DOCTYPE html>\n<html class="desktop ">\n <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">\n  <title>\n   1984 by George Orwell\n  </title>\n  <script type="text/javascript">\n   if (window.Mobvious === undefined) {\n          window.Mobvious = {};\n        }\n        window.Mobvious.device_type = \'desktop\';\n  </script>\n  <script src="https://s.gr-assets.com/assets/webfontloader-d12a76562d1fabc96ea2ca165a5f46ae.js">\n  </script>\n  <script>\n   //<![CDATA[\n\n  WebFont.load({\n    classes: false,\n    custom: {\n      families: ["Lato:n4,n7,i4", "Merriweather:n4,n7,i4"],\n      urls: ["https://s.gr-assets.com/assets/gr/fonts-e256f84093cc13b27f5b82343398031a.css"]\n    }\n  });\n\n//]]>\n  </script>\n  <link href="https://s.gr-assets.com/assets/goodreads-bad0777cf7270fb59d36c8f13c6d96b9.css" media="all" rel="stylesheet"/>\n  <!--[if lte IE 9]>\n<link rel="stylesheet" media="all" href="https://s.gr-assets.com/assets/goodread

In [0]:
d.select("meta[property='og:title']")[0]['content']

'1984'

Lets get everything we want...

In [0]:
d=BeautifulSoup(fstuff.text, 'html.parser')
print(
"title", d.select_one("meta[property='og:title']")['content'],"\n",
"isbn", d.select("meta[property='books:isbn']")[0]['content'],"\n",
"type", d.select("meta[property='og:type']")[0]['content'],"\n",
"author", d.select("meta[property='books:author']")[0]['content'],"\n",
"average rating", d.select_one("span.average").text,"\n",
"ratingCount", d.select("meta[itemprop='ratingCount']")[0]["content"],"\n",
"reviewCount", d.select_one("span.count")["title"]
)

title 1984 
 isbn 9780451524935 
 type books.book 
 author https://www.goodreads.com/author/show/3706.George_Orwell 
 average rating 4.16 
 ratingCount 2402209 
 reviewCount 53335


Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [0]:
"list{:0>2}.txt".format(3)

'list03.txt'

In [0]:
a = "4"
b = 4
class Four:
    def __str__(self):
        return "Fourteen"
c=Four()

In [0]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

'The hazy cat jumped over the 4 and 4 and Fourteen'

## 4. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [0]:
fetched=[]
for i in range(1,3):
    with open("files/list{:0>2}.txt".format(i)) as fd:
        counter=0
        for bookurl_line in fd:
            if counter > 4:
                break
            bookurl=bookurl_line.strip()
            stuff=requests.get(URLSTART+bookurl)
            filetowrite=bookurl.split('/')[-1]
            filetowrite="files/"+str(i)+"_"+filetowrite+".html"
            print("FTW", filetowrite)
            fd=open(filetowrite,"w", encoding='utf-8')
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite)
            time.sleep(2)
            counter=counter+1
            
print(fetched)

FTW files/1_2767052-the-hunger-games.html
FTW files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
FTW files/1_2657.To_Kill_a_Mockingbird.html
FTW files/1_1885.Pride_and_Prejudice.html
FTW files/1_41865.Twilight.html
FTW files/2_5470.1984.html
FTW files/2_4989.The_Red_Tent.html
FTW files/2_37435.The_Secret_Life_of_Bees.html
FTW files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
FTW files/2_7171637-clockwork-angel.html
['files/1_2767052-the-hunger-games.html', 'files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html', 'files/1_2657.To_Kill_a_Mockingbird.html', 'files/1_1885.Pride_and_Prejudice.html', 'files/1_41865.Twilight.html', 'files/2_5470.1984.html', 'files/2_4989.The_Red_Tent.html', 'files/2_37435.The_Secret_Life_of_Bees.html', 'files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html', 'files/2_7171637-clockwork-angel.html']


Ok we are off to parse each one of the html pages we fetched. We have provided the skeleton of the code and the code to parse the year, since it is a bit more complex...see the difference in the screenshots above. 

In [0]:
import re
yearre = r'\d{4}'
def get_year(d):
    if d.select_one("nobr.greyText"):
        return d.select_one("nobr.greyText").text.strip().split()[-1][:-1]
    else:
        thetext=d.select("div#details div.row")[1].text.strip()
        rowmatch=re.findall(yearre, thetext)
        if len(rowmatch) > 0:
            rowtext=rowmatch[0].strip()
        else:
            rowtext="NA"
        return rowtext

<div class="exercise"><b>Exercise</b></div>

Your job is to fill in the code to get the genres.

In [0]:
def get_genres(d):
    # your code here
    genres=d.select("div.elementList div.left a")
    glist=[]
    for g in genres:
        glist.append(g['href'])
    return glist

In [0]:

listofdicts=[]
for filetoread in fetched:
    print(filetoread)
    td={}
    with open(filetoread) as fd:
        datext = fd.read()
    d=BeautifulSoup(datext, 'html.parser')
    td['title']=d.select_one("meta[property='og:title']")['content']
    td['isbn']=d.select_one("meta[property='books:isbn']")['content']
    td['booktype']=d.select_one("meta[property='og:type']")['content']
    td['author']=d.select_one("meta[property='books:author']")['content']
    td['rating']=d.select_one("span.average").text
    td['ratingCount']=d.select_one("meta[itemprop='ratingCount']")["content"]
    td['reviewCount']=d.select_one("span.count")["title"]
    td['year'] = get_year(d)
    td['file']=filetoread
    glist = get_genres(d)
    td['genres']="|".join(glist)
    listofdicts.append(td)

files/1_2767052-the-hunger-games.html
files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
files/1_2657.To_Kill_a_Mockingbird.html
files/1_1885.Pride_and_Prejudice.html
files/1_41865.Twilight.html
files/2_5470.1984.html
files/2_4989.The_Red_Tent.html
files/2_37435.The_Secret_Life_of_Bees.html
files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
files/2_7171637-clockwork-angel.html


In [0]:
listofdicts[0]

{'title': 'The Hunger Games (The Hunger Games, #1)',
 'isbn': '9780439023481',
 'booktype': 'books.book',
 'author': 'https://www.goodreads.com/author/show/153394.Suzanne_Collins',
 'rating': '4.33',
 'ratingCount': '5491176',
 'reviewCount': '160373',
 'year': '2008',
 'file': 'files/1_2767052-the-hunger-games.html',
 'genres': '/genres/young-adult|/genres/fiction|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction'}

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [0]:
df = pd.DataFrame.from_records(listofdicts)
df.head()

,author,booktype,file,genres,isbn,rating,ratingCount,reviewCount,title,year
0,https://www.goodreads.com/author/show/153394.S...,books.book,files/1_2767052-the-hunger-games.html,/genres/young-adult|/genres/fiction|/genres/sc...,9780439023481,4.33,5491176,160373,"The Hunger Games (The Hunger Games, #1)",2008
1,https://www.goodreads.com/author/show/1077326....,books.book,files/1_2.Harry_Potter_and_the_Order_of_the_Ph...,/genres/fantasy|/genres/young-adult|/genres/fi...,9780439358071,4.48,2030257,33033,Harry Potter and the Order of the Phoenix (Har...,2003
2,https://www.goodreads.com/author/show/1825.Har...,books.book,files/1_2657.To_Kill_a_Mockingbird.html,/genres/classics|/genres/fiction|/genres/histo...,9780061120084,4.27,3722962,79058,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",1960
3,https://www.goodreads.com/author/show/1265.Jan...,books.book,files/1_1885.Pride_and_Prejudice.html,/genres/classics|/genres/fiction|/genres/romance,9780679783268,4.25,2438138,54013,Pride and Prejudice,1813
4,https://www.goodreads.com/author/show/941441.S...,books.book,files/1_41865.Twilight.html,/genres/young-adult|/genres/fantasy|/genres/ro...,9780316015844,3.58,4262416,97797,"Twilight (Twilight, #1)",2005


In [0]:
df.to_csv("files/meta.csv", index=False, header=True)